Honestly, I just cannot understand training with teamID which is unexplanable! <p>
Just imagine team0 won team5 and team2 won team5, could you tell that team 1 will win team5 as well? <p>
It would have a good performance on training data, but poor on prediction.<p>
Anyway, let's see how it works?

# Train by season and teamID within 2015-2019
This is definitely tricky, we are training on what we are predicting :)

In [ ]:
import pandas as pd

In [ ]:
df_R = pd.read_csv('../input/ncaam-march-mania-2021/MNCAATourneyDetailedResults.csv',sep=',',header=0,index_col=None)
df_R.drop(df_R.loc[(df_R['Season']<2015)|(df_R['Season']>2019),:].index,inplace=True)
df_R

In [ ]:
df_sub = pd.read_csv('../input/ncaam-march-mania-2021/MSampleSubmissionStage1.csv',sep=',',header=0,index_col=None)
df_sub

Train by ID

In [ ]:
df_Rid_1 = df_R[['Season','WTeamID','LTeamID']]
df_Rid_1['Label']=1
df_Rid_1

df_Rid_0 = df_R[['Season','LTeamID','WTeamID']]
df_Rid_0.columns = ['Season','WTeamID','LTeamID']
df_Rid_0['Label']=0

df_all = pd.concat([df_Rid_1,df_Rid_0])
df_all

In [ ]:
import numpy as np
import pandas as pd
import pickle
import xgboost as xgb
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report


In [ ]:
train, test = train_test_split(df_all)

In [ ]:
train_X = train.drop('Label', axis=1).values
train_y = train['Label'].values
test_X = test.drop('Label', axis=1).values
test_y = test['Label'].values


In [ ]:
clf = GridSearchCV(
    estimator = XGBClassifier(use_label_encoder =False,verbosity=0),
    param_grid = {'booster':['gbtree', 'gblinear', 'dart'],
        'n_estimators': [2,5,10, 20, 50],
                  'max_depth': [3,4,5,6,10,20],
                  'subsample':[0.6, 0.7, 0.8]},
    cv=StratifiedKFold(n_splits=5).split(train_X, train_y)
)
clf.fit(train_X, train_y)

In [ ]:
print(clf.best_params_)
print(classification_report(clf.best_estimator_.predict(test_X), test_y))

In [ ]:
df_sub['Season'] = df_sub['ID'].apply(lambda x: x.split('_')[0])
df_sub['Team1'] = df_sub['ID'].apply(lambda x: x.split('_')[1])
df_sub['Team2'] = df_sub['ID'].apply(lambda x: x.split('_')[2])
df_sub

In [ ]:
predict_features = df_sub.drop(['ID','Pred'], axis=1).values

df_final = df_sub
df_final['Pred'] = clf.predict_proba(predict_features)[:,1]


In [ ]:
df_final[['ID','Pred']]
df_final[['ID','Pred']].set_index('ID').to_csv('submissionID1.csv')

# Train by teamID only within 2015-2019

In [ ]:
df_all2 = df_all.drop('Season',axis=1)

In [ ]:
train, test = train_test_split(df_all2)

In [ ]:
train_X = train.drop('Label', axis=1).values
train_y = train['Label'].values
test_X = test.drop('Label', axis=1).values
test_y = test['Label'].values

In [ ]:
clf = GridSearchCV(
    estimator = XGBClassifier(use_label_encoder =False,verbosity=0),
    param_grid = {'booster':['gbtree', 'gblinear', 'dart'],
        'n_estimators': [2,5,10, 20, 50],
                  'max_depth': [3,4,5,6,10,20],
                  'subsample':[0.6, 0.7, 0.8]},
    cv=StratifiedKFold(n_splits=5).split(train_X, train_y)
)
clf.fit(train_X, train_y)

In [ ]:
print(clf.best_params_)
print(classification_report(clf.best_estimator_.predict(test_X), test_y))

In [ ]:
predict_features = df_sub.drop(['ID','Pred','Season'], axis=1).values

df_final = df_sub
df_final['Pred'] = clf.predict_proba(predict_features)[:,1]

In [ ]:
df_final[['ID','Pred']]
df_final[['ID','Pred']].set_index('ID').to_csv('submissionID2.csv')

# Conclusion
The prediction scores are good, with the best 0.47885 and 0.52537.
After all training on ID is like a forced remember, just probably why the 3rd prize last competition is based on teamID with a little luck. 